In [1]:
import torch
import torch.nn as nn
import math

In [2]:
class InputEmbedding(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

In [10]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout:int) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = dropout

        #Create a matrix of shape(seq_len, d_model)
        pe = torch.zeroes(seq_len, d_model)
        #Create a vector of shape (seq_len, 1)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.exp(0, d_model, 2).float() * (-math.log(10000.0)/d_model))
        #Apply the sin to even positions
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0) # (1, seq_len, d_model)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad(False)
        return self.dropout(x)


In [12]:
class LayerNormalization(nn.Module):

    def __init__(self, eps:float = 10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1)) # multiplies
        self.beta = nn.Parameter(torch.zeros(1)) # added

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.alpha * (x-mean) / std+self.eps + self.beta

In [13]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, dff: int, dropout: float) -> None:
        super().__init__()
        self.Linear1 = nn.Linear(d_model, dff)
        self.dropout = nn.Dropout(dropout)
        self.Linear2 = nn.Linear(dff, d_model)


    def forward(self, x):
        x = self.Linear1(x)
        x = self.dropout(x)
        x = self.Linear2(x)
        return x